<a href="https://colab.research.google.com/github/colinjhorgan/MLTSAFinalProject/blob/main/Preprocessing/CNN_Preprocessing_Spherical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as plt
%pylab inline

from scipy.signal import resample_poly

from google.colab import drive
drive.mount('/content/gdrive')

import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Mounted at /content/gdrive


In [ ]:
#Read in data
path = '/content/gdrive/MyDrive/3. MSDS Courses/Machine Learning Time Series/MLTSA22 Project/Data/Phones_accelerometer.csv'
paccel = pd.read_csv(path, index_col=0)

paccel.tail(8)

,Arrival_Time,Creation_Time,x,y,z,User,Model,Device,gt
Index,,,,,,,,,
129045,1424778553265,92263731712000,1.379043,0.306454,9.959755,i,samsungold,samsungold_2,bike
129046,1424778553286,92263752250000,1.532270,0.153227,9.959755,i,samsungold,samsungold_2,bike
129047,1424778553306,92263772270000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike
129048,1424778553315,92263781761000,1.379043,0.153227,9.959755,i,samsungold,samsungold_2,bike
129049,1424778553346,92263812248000,1.379043,0.153227,9.806528,i,samsungold,samsungold_2,bike
129050,1424778553366,92263832267000,1.532270,0.153227,9.806528,i,samsungold,samsungold_2,bike
129051,1424778553386,92263852409000,1.532270,0.000000,9.959755,i,samsungold,samsungold_2,bike
129052,1424778553395,92263861839000,1.379043,0.000000,9.959755,i,samsungold,samsungold_2,bike


In [ ]:
#gt contains nan values so we impute them with 'no_task'
paccel['gt'] = paccel['gt'].fillna('no_task')

In [ ]:
# get x,y,z values
x = paccel["x"].values
y = paccel["y"].values
z = paccel["z"].values

In [ ]:
# cartesian to spherical
p = np.sqrt(x**2 + y**2 + z**2)
theta = np.arctan2(y,x)
phi = np.arctan2(np.sqrt(x**2 + y**2),z)

In [ ]:
# set spherical coordinates
paccel["rho"] = p
paccel["theta"] = theta
paccel["phi"] = phi

In [ ]:
# resample to 15k
rho_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['rho'].apply(lambda rho : resample_poly(rho.values, 15050, rho.values.shape[0]))
theta_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['theta'].apply(lambda theta : resample_poly(theta.values, 15050, theta.values.shape[0]))
phi_fixed = paccel.reset_index().groupby(['User','Model','Device','gt'])['phi'].apply(lambda phi : resample_poly(phi.values, 15050, phi.values.shape[0]))

In [ ]:
# make new df
new_df = rho_fixed.to_frame()
new_df = new_df.merge(theta_fixed.to_frame(), left_index=True, right_index=True)
new_df = new_df.merge(phi_fixed.to_frame(), left_index=True, right_index=True)

In [ ]:
#Standardize each series, and apply sliding window with length 100 and 50% overlap

#sig = lambda x : 1/(1 + np.exp(-x)) # sigmoid function

images = []
for i in range(new_df.shape[0]):
    x = new_df.iloc[[i]].rho.values[0]
    y = new_df.iloc[[i]].theta.values[0]
    z = new_df.iloc[[i]].phi.values[0]

    x = ((x - x.min() ) / (x.max() - x.min()))
    y = ((y - y.min() ) / (y.max() - y.min()))
    z = ((z - z.min() ) / (z.max() - z.min()))
    
    r = np.stack([x[i:100+i] for i in range(0, 15000, 50)]) # stack elements using 100-sample window with 50% overlap
    g = np.stack([y[i:100+i] for i in range(0, 15000, 50)])
    b = np.stack([z[i:100+i] for i in range(0, 15000, 50)])
    
    vals = np.dstack([r,g,b])
    images.append(vals)
    
im_arr = np.array(images)

In [ ]:
im_arr.shape #(num_samples, height, length, num_layers)

(485, 300, 100, 3)

In [ ]:
np.save('/content/gdrive/MyDrive/3. MSDS Courses/Machine Learning Time Series/MLTSA22 Project/Data/image_array_polar.npy', im_arr)